In [0]:
# 04_data_analysis_and_insights.ipynb - Célula 1: Configuração da SparkSession

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, from_unixtime, to_date, when, concat_ws, transform, sum as spark_sum, count as spark_count, avg as spark_avg

print("Bibliotecas importadas com sucesso.")

# A SparkSession já é automaticamente fornecida e configurada pelo Databricks.
# Ela já herda o acesso ao S3 via a IAM Role do seu CloudFormation.
spark = SparkSession.builder.appName("TFTDataAnalysisAndInsights").getOrCreate()

print("SparkSession configurada.")

In [0]:
# 04_data_analysis_and_insights.ipynb - Célula 2: Carregar Dados Transformados (Tabela Delta)

from pyspark.sql.functions import col

# Nome completo da sua tabela Delta na camada 'curated' do Unity Catalog
UNITY_CATALOG_DELTA_TABLE_NAME = "tft_analytics.curated.tft_player_performance_delta"

print(f"Carregando dados da tabela curada '{UNITY_CATALOG_DELTA_TABLE_NAME}'...")

# Carrega o DataFrame a partir da tabela Delta do Unity Catalog
df_curated = spark.read.table(UNITY_CATALOG_DELTA_TABLE_NAME)

print(f"Dados curados carregados. Total de registros: {df_curated.count()}")
print("\nEsquema dos dados curados:")
df_curated.printSchema()

print("\nExemplo das primeiras 5 linhas dos dados curados:")
display(df_curated.limit(5))

In [0]:
# 04_data_analysis_and_insights.ipynb - Célula 3: Análises de Desempenho Básico

from pyspark.sql.functions import count, sum, avg, round, lit

print("Iniciando análises de desempenho básico...")

# Seu PUUID real para filtrar suas partidas
MY_PUUID = "pPlTg-W4Us3Owj5jTOelDd650T-gA40TUuwdptOo1AhNgMhVkEFK75O-sfYEqIGp2cn_XVJjaYE2Zg"

# 1. Converter o DataFrame para uma View SQL temporária para facilitar consultas SQL
df_curated.createOrReplaceTempView("tft_player_performance")

# 2. Análise Geral de Colocação (filtrando pelo seu PUUID)
print("\n--- Análise Geral de Colocação ---")
spark.sql(f"""
    SELECT
        COUNT(*) AS total_games_played,
        SUM(CASE WHEN placement = 1 THEN 1 ELSE 0 END) AS num_first_places,
        ROUND((SUM(CASE WHEN placement = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS first_place_rate_percent,
        SUM(CASE WHEN placement <= 4 THEN 1 ELSE 0 END) AS num_top4_places,
        ROUND((SUM(CASE WHEN placement <= 4 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS top4_rate_percent,
        ROUND(AVG(placement), 2) AS average_placement
    FROM
        tft_player_performance
    WHERE
        puuid = '{MY_PUUID}' -- Usando seu PUUID
""").show()

# 3. Análise de Colocação por Versão do Jogo (Patch), filtrando pelo seu PUUID
print("\n--- Análise de Colocação por Versão do Jogo (Patch) ---")
spark.sql(f"""
    SELECT
        game_version,
        COUNT(*) AS total_games,
        ROUND(AVG(placement), 2) AS average_placement,
        SUM(CASE WHEN placement = 1 THEN 1 ELSE 0 END) AS num_first_places,
        ROUND((SUM(CASE WHEN placement = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS first_place_rate_percent,
        SUM(CASE WHEN placement <= 4 THEN 1 ELSE 0 END) AS num_top4_places,
        ROUND((SUM(CASE WHEN placement <= 4 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) AS top4_rate_percent
    FROM
        tft_player_performance
    WHERE
        puuid = '{MY_PUUID}' -- Usando seu PUUID
    GROUP BY
        game_version
    ORDER BY
        average_placement ASC
""").show(truncate=False)

# 4. Distribuição das Colocações, filtrando pelo seu PUUID
print("\n--- Distribuição das Colocações ---")
spark.sql(f"""
    SELECT
        placement,
        COUNT(*) AS num_games,
        ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM tft_player_performance WHERE puuid = '{MY_PUUID}')), 2) AS percentage
    FROM
        tft_player_performance
    WHERE
        puuid = '{MY_PUUID}' -- Usando seu PUUID
    GROUP BY
        placement
    ORDER BY
        placement ASC
""").show()